In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [2]:
simus = ['ssp126',
         'ssp370',
         'ssp585',
         'hist-GHG',
         'hist-aer']
data_path = "/Users/chubb/DSC_180A/ClimateBench/data_preprocessed/train_val/"

In [3]:
X_train = []
Y_train = []

for i, simu in enumerate(simus):

    input_name = 'inputs_' + simu + '.nc'
    output_name = 'outputs_' + simu + '.nc'

    # Just load hist data in these cases 'hist-GHG' and 'hist-aer'
    if 'hist' in simu:
        # load inputs 
        input_xr = xr.open_dataset(data_path + input_name)
            
        # load outputs                                                             
        output_xr = xr.open_dataset(data_path + output_name).mean(dim='member')
        output_xr = output_xr.assign({"pr": output_xr.pr * 86400,
                                      "pr90": output_xr.pr90 * 86400}).rename({'lon':'longitude', 
                                                                               'lat': 'latitude'}).transpose('time','latitude', 'longitude').drop(['quantile'])
    
    # Concatenate with historical data in the case of scenario 'ssp126', 'ssp370' and 'ssp585'
    else:
        # load inputs 
        input_xr = xr.open_mfdataset([data_path + 'inputs_historical.nc', 
                                    data_path + input_name]).compute()
            
        # load outputs                                                             
        output_xr = xr.concat([xr.open_dataset(data_path + 'outputs_historical.nc').mean(dim='member'),
                               xr.open_dataset(data_path + output_name).mean(dim='member')],
                               dim='time').compute()
        output_xr = output_xr.assign({"pr": output_xr.pr * 86400,
                                      "pr90": output_xr.pr90 * 86400}).rename({'lon':'longitude', 
                                                                               'lat': 'latitude'}).transpose('time','latitude', 'longitude').drop(['quantile'])

    print(input_xr.dims, simu)
        # Append to list 
    X_train.append(input_xr)
    Y_train.append(output_xr)   

Frozen({'time': 251, 'longitude': 144, 'latitude': 96}) ssp126
Frozen({'time': 251, 'longitude': 144, 'latitude': 96}) ssp370
Frozen({'time': 251, 'longitude': 144, 'latitude': 96}) ssp585
Frozen({'time': 165, 'longitude': 144, 'latitude': 96}) hist-GHG
Frozen({'time': 165, 'longitude': 144, 'latitude': 96}) hist-aer


In [4]:
def normalize(data, var, meanstd_dict):
    mean = meanstd_dict[var][0]
    std = meanstd_dict[var][1]
    return (data - mean)/std

def unnormalize(data, var, meanstd_dict):
    mean = meanstd_dict[var][0]
    std = meanstd_dict[var][1]
    return data * std + mean
len_historical = 165

In [5]:
# Compute mean/std of each variable for the whole dataset
meanstd_inputs = {}

for var in ['CO2', 'CH4', 'SO2', 'BC']:
    # To not take the historical data into account several time we have to slice the scenario datasets
    # and only keep the historical data once (in the first ssp index 0 in the simus list)
    array = np.concatenate([X_train[i][var].data for i in [0, 3, 4]] + 
                           [X_train[i][var].sel(time=slice(len_historical, None)).data for i in range(1, 3)])
    print((array.mean(), array.std()))
    meanstd_inputs[var] = (array.mean(), array.std())

(1074.172303244536, 1755.690699230666)
(0.1927369743762821, 0.18457590641432994)
(2.5623359997066755e-12, 2.250114566783271e-11)
(1.4947905009818064e-13, 1.0313342554838387e-12)


In [6]:
X_train_norm = [] 
for i, train_xr in enumerate(X_train): 
    for var in ['CO2', 'CH4', 'SO2', 'BC']: 
        var_dims = train_xr[var].dims
        train_xr=train_xr.assign({var: (var_dims, normalize(train_xr[var].data, var, meanstd_inputs))}) 
    X_train_norm.append(train_xr)

In [7]:
slider = 10 # years moving temporal window 
# Functions for reshaping the data 
def input_for_training(X_train_xr, skip_historical=False, len_historical=None): 
    
    X_train_np =  X_train_xr.to_array().transpose('time', 'latitude', 'longitude', 'variable').data

    time_length = X_train_np.shape[0]
    # If we skip historical data, the first sequence created has as last element the first scenario data point
    if skip_historical:
        X_train_to_return = np.array([X_train_np[i:i+slider] for i in range(len_historical-slider+1, time_length-slider+1)])
    # Else we just go through the whole dataset historical + scenario (does not matter in the case of 'hist-GHG' and 'hist_aer')
    else:
        X_train_to_return = np.array([X_train_np[i:i+slider] for i in range(0, time_length-slider+1)])
    
    return X_train_to_return 


def output_for_training(Y_train_xr, var, skip_historical=False, len_historical=None): 
    Y_train_np = Y_train_xr[var].data
    
    time_length = Y_train_np.shape[0]
    
    # If we skip historical data, the first sequence created has as target element the first scenario data point
    if skip_historical:
        Y_train_to_return = np.array([[Y_train_np[i+slider-1]] for i in range(len_historical-slider+1, time_length-slider+1)])
    # Else we just go through the whole dataset historical + scenario (does not matter in the case of 'hist-GHG' and 'hist_aer')
    else:
        Y_train_to_return = np.array([[Y_train_np[i+slider-1]] for i in range(0, time_length-slider+1)])
    
    return Y_train_to_return

In [8]:
var_to_predict =  'tas'
# skip_historical set to (i < 2) because of the order of the scenario and historical runs in the X_train and Y_train lists.
# In details: ssp126 0, ssp370 1 = skip historical part of the data, ssp585 2, hist-GHG 3 and hist-aer 4 = keep the whole sequence
X_train_all = np.concatenate([input_for_training(X_train_norm[i], skip_historical=(i<2), len_historical=len_historical) for i in range(len(simus))], axis = 0)
Y_train_all = np.concatenate([output_for_training(Y_train[i], var_to_predict, skip_historical=(i<2), len_historical=len_historical) for i in range(len(simus))], axis=0)
print(X_train_all.shape)
print(Y_train_all.shape)

(726, 10, 96, 144, 4)
(726, 1, 96, 144)


In [50]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, Flatten, Input, Reshape, AveragePooling2D, MaxPooling2D, Conv2DTranspose, TimeDistributed, LSTM, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.regularizers import l2

import random 
seed = 6 
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [83]:
class PINN(tf.Module):
    def __init__(self, X_train, Y_train, X_test, slider=10):
        super().__init__()
        self.slider = slider
        self.input_train = X_train
        self.output_train = Y_train
        self.test = X_test
        self.model = self.build_model()

    def build_model(self):
        cnn_model = Sequential()
        cnn_model.add(Input(shape=(self.slider, 96, 144, 4)))
        cnn_model.add(TimeDistributed(Conv2D(20, (3, 3), padding='same', activation='relu'), input_shape=(self.slider, 96, 144, 4)))
        cnn_model.add(TimeDistributed(AveragePooling2D(2)))
        cnn_model.add(TimeDistributed(GlobalAveragePooling2D()))
        cnn_model.add(LSTM(25, activation='relu'))
        cnn_model.add(Dense(1*96*144))
        cnn_model.add(Activation('linear'))
        cnn_model.add(Reshape((1, 96, 144)))
        return cnn_model

    def compile_model(self, optimizer='rmsprop', metrics=None):
        self.model.compile(optimizer=optimizer, loss=self.PINN_loss, metrics=metrics)

    def summary(self):
        self.model.summary()

    def PINN_loss(self, y_t, y_p):
        error = y_t - y_p
        # MSE between training and prediction
        mse = tf.reduce_mean(tf.square(error))
        # FaIR loss
        C0 = 598
        E = self.test["CO2"].sel(time=slice("1859", "2100")).data
        C = [C0 + E[i] for i in range(len(E))]
        F = np.multiply(4, np.log(np.divide(C,C0)))
        qj = 1
        T = tf.constant(np.multiply(qj, F), dtype=tf.float32)
        fair= T - y_p
        fair_mse = tf.reduce_mean(tf.square(fair))
        total = mse + fair_mse
        return total

    
    def fit(self, use_multiprocessing=True, batch_size=10, epochs=30, verbose=1):
        self.model.fit(self.input_train, self.output_train, use_multiprocessing=use_multiprocessing, batch_size=batch_size, epochs=epochs, verbose=verbose)


In [84]:
X_test = xr.open_mfdataset([data_path + 'inputs_historical.nc',
                            data_path + 'inputs_ssp245.nc']).compute()
model = PINN(X_train=X_train_all, Y_train=Y_train_all, X_test=X_test)
model.compile_model()

In [ ]:
model.fit()

Epoch 1/30
Tensor("PINN_loss/Const_1:0", shape=(242,), dtype=float32)


ValueError: in user code:

    File "c:\Users\chubb\anaconda3\envs\ClimateBench\Lib\site-packages\keras\src\engine\training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\chubb\AppData\Local\Temp\ipykernel_37748\1443012157.py", line 41, in PINN_loss  *
        y_p = tf.reshape(y_p, (242,))

    ValueError: Dimension size must be evenly divisible by 13824 but is 242 for '{{node PINN_loss/Reshape}} = Reshape[T=DT_FLOAT, Tshape=DT_INT32](sequential_17/reshape_17/Reshape, PINN_loss/Reshape/shape)' with input shapes: [?,1,96,144], [1] and with input tensors computed as partial shapes: input[1] = [242].


In [88]:
C0 = 598
E = X_test["CO2"].sel(time=slice("1859", "2100")).data
C = [C0 + E[i] for i in range(len(E))]
F = np.multiply(4, np.log(np.divide(C,C0)))
qj = 1
T = np.multiply(qj, F)
X_test

<xarray.Dataset>
Dimensions:    (time: 251, longitude: 144, latitude: 96)
Coordinates:
  * time       (time) int64 1850 1851 1852 1853 1854 ... 2097 2098 2099 2100
  * longitude  (longitude) float64 0.0 2.5 5.0 7.5 ... 350.0 352.5 355.0 357.5
  * latitude   (latitude) float64 -90.0 -88.11 -86.21 ... 86.21 88.11 90.0
Data variables:
    CO2        (time) float64 0.1883 0.3772 0.5738 ... 4.51e+03 4.524e+03
    SO2        (time, latitude, longitude) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    CH4        (time) float64 0.03131 0.03174 0.03218 ... 0.281 0.2807 0.2803
    BC         (time, latitude, longitude) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0

In [16]:
# Normalize data 
for var in ['CO2', 'CH4', 'SO2', 'BC']: 
    var_dims = X_test[var].dims
    X_test = X_test.assign({var: (var_dims, normalize(X_test[var].data, var, meanstd_inputs))}) 
    
X_test_np = input_for_training(X_test, skip_historical=False, len_historical=len_historical)

In [26]:
m_pred = model.model.predict(X_test_np)
print(m_pred.shape)
m_pred = m_pred.reshape(m_pred.shape[0], m_pred.shape[2], m_pred.shape[3])
m_pred = xr.DataArray(m_pred, dims=['time', 'lat', 'lon'], coords=[X_test.time.data[slider-1:], X_test.latitude.data, X_test.longitude.data])
m_pred = m_pred.transpose('lat', 'lon', 'time').to_dataset(name=var_to_predict)
m_pred

8/8 [==============================] - 1s 172ms/step
(242, 1, 96, 144)


<xarray.Dataset>
Dimensions:  (time: 242, lat: 96, lon: 144)
Coordinates:
  * time     (time) int64 1859 1860 1861 1862 1863 ... 2096 2097 2098 2099 2100
  * lat      (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 84.32 86.21 88.11 90.0
  * lon      (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Data variables:
    tas      (lat, lon, time) float32 0.00296 0.002956 ... 0.01405 0.01412